Movie Recommendation System

In [ ]:
import os
import plotly.express as px
import numpy as np
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
import re
from sklearn.neighbors import NearestNeighbors
import random
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
!pip install mlxtend

In [ ]:
dataNames = pd.read_csv("/content/movies.csv")
dataRatings = pd.read_csv("/content/ratings.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
dataRatings.head()  # First dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating,timestamp
0,1,110,1.0,1.425942e+09
1,1,147,4.5,1.425942e+09
2,1,858,5.0,1.425942e+09
3,1,1221,5.0,1.425942e+09
4,1,1246,5.0,1.425942e+09


In [ ]:
dataNames.head()  # Second dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Inner join of two datasets (common column is movieId)
data = pd.merge(dataRatings, dataNames, how='inner')
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating,timestamp,title,genres
0,1,110,1.0,1.425942e+09,Braveheart (1995),Action|Drama|War
1,11,110,3.5,1.231677e+09,Braveheart (1995),Action|Drama|War
2,22,110,5.0,1.111937e+09,Braveheart (1995),Action|Drama|War
3,24,110,5.0,9.798700e+08,Braveheart (1995),Action|Drama|War
4,29,110,3.0,1.044020e+09,Braveheart (1995),Action|Drama|War


In [ ]:
# Number of Users who rated at least one movie:
print("Number of Users who rated at least one move: ", data.userId.nunique())
print("-"*25)

# Number of Movies in the dataset:
print("Number of Movies in the dataset:", data.title.nunique())
print("-"*25)

# Unique of Rating points in the dataset:
print("Unique Rating points:", data.rating.unique())

Number of Users who rated at least one move:  853
-------------------------
Number of Movies in the dataset: 6562
-------------------------
Unique Rating points: [1.  3.5 5.  3.  4.  0.5 4.5 2.  2.5 1.5 nan]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Extracting movie release years into one column
data['movie_year'] = data.title.str.extract('.*\((.*)\).*')
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:2: DeprecationWarning: invalid escape sequence '\('
<>:2: DeprecationWarning: invalid escape sequence '\('
<ipython-input-8-259254d3619d>:2: DeprecationWarning: invalid escape sequence '\('
  data['movie_year'] = data.title.str.extract('.*\((.*)\).*')


,userId,movieId,rating,timestamp,title,genres,movie_year
0,1,110,1.0,1.425942e+09,Braveheart (1995),Action|Drama|War,1995
1,11,110,3.5,1.231677e+09,Braveheart (1995),Action|Drama|War,1995
2,22,110,5.0,1.111937e+09,Braveheart (1995),Action|Drama|War,1995
3,24,110,5.0,9.798700e+08,Braveheart (1995),Action|Drama|War,1995
4,29,110,3.0,1.044020e+09,Braveheart (1995),Action|Drama|War,1995


In [ ]:
# Removing year from the movie titles
data['title'] = data.title.str.split('(').str[0].str[:-1]
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating,timestamp,title,genres,movie_year
0,1,110,1.0,1.425942e+09,Braveheart,Action|Drama|War,1995
1,11,110,3.5,1.231677e+09,Braveheart,Action|Drama|War,1995
2,22,110,5.0,1.111937e+09,Braveheart,Action|Drama|War,1995
3,24,110,5.0,9.798700e+08,Braveheart,Action|Drama|War,1995
4,29,110,3.0,1.044020e+09,Braveheart,Action|Drama|War,1995


In [ ]:
def UNIX_to_Readable(df):
    if pd.notna(df):
        return pd.to_datetime(datetime.fromtimestamp(df).strftime('%Y-%m-%d %H:%M:%S'))
    else:
        return pd.NaT  # Return NaT (Not a Time) for NaN values

# Converting Unix date-format to readable format, handling NaN values
data.timestamp = data.timestamp.apply(UNIX_to_Readable)

# Display the updated DataFrame
print(data.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   userId  movieId  rating           timestamp       title            genres  \
0       1      110     1.0 2015-03-09 22:52:09  Braveheart  Action|Drama|War   
1      11      110     3.5 2009-01-11 12:29:49  Braveheart  Action|Drama|War   
2      22      110     5.0 2005-03-27 15:23:29  Braveheart  Action|Drama|War   
3      24      110     5.0 2001-01-19 02:06:52  Braveheart  Action|Drama|War   
4      29      110     3.0 2003-01-31 13:33:25  Braveheart  Action|Drama|War   

  movie_year  
0       1995  
1       1995  
2       1995  
3       1995  
4       1995  


In [ ]:
# Removing decimal values to the ceiling value to decrease number of rating classes
data.rating = np.ceil(data.rating)
print("Unique Rating Points:", data.rating.unique())

Unique Rating Points: [ 1.  4.  5.  3.  2. nan]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
_ = data.title.value_counts()
fig = px.histogram(_, x=_, opacity=0.85, marginal='box',
                   labels={
                       'x': 'Number of Ratings taken'})
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Distribution of the Number of Ratings taken by the Movies',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Removing movies that rated less than 10
movieFrequency_greater_10 = data['movieId'].value_counts()[data['movieId'].value_counts() >= 10].index
data = data[data.movieId.isin(movieFrequency_greater_10)]

print("Minimum Number of Rated Movies after Drop:", data.title.value_counts().nsmallest(5))   # So we achieved to obtain number of ratings taken by users minimum 10

Minimum Number of Rated Movies after Drop: Lolita                                         10
Milk                                           10
Indecent Proposal                              10
Talladega Nights: The Ballad of Ricky Bobby    10
Girl Next Door, The                            10
Name: title, dtype: int64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Exploratory Data Analysis

Frequency of the Ratings

In [ ]:
rating_val_count = data.rating.value_counts()
fig = px.bar(rating_val_count, x=rating_val_count.index, y=rating_val_count, text=rating_val_count,
             labels={
                 "index": "Ratings",
                 'y': 'Number of Ratings'},
             color=rating_val_count
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Frequency of the Ratings',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Top Frequent the Movie Genres

In [ ]:
genres_value_counts = data['genres'].str.split('|', expand=True).stack().value_counts()
fig = px.bar(genres_value_counts, x=genres_value_counts.index, y=genres_value_counts, text=genres_value_counts,
             labels={
                 "index": "Genres",
                 'y': 'Frequency'},
             color=genres_value_counts
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top Frequent the Movie Genres',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Number of Movies for each Genre


In [ ]:
unique_movies = data.drop_duplicates('title')
unique_movies = unique_movies['genres'].str.split('|', expand=True).stack().value_counts()
fig = px.bar(unique_movies, x=unique_movies.index, y=unique_movies, text=unique_movies,
             labels={
                 "index": "Genres",
                 'y': 'Number of Movies'},
             color=unique_movies
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Number of Movies for each Genre',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
data['timeDifferenceAfterRelease'] = data.timestamp.dt.year - data.movie_year.astype(int)
fig = px.histogram(data, x='timeDifferenceAfterRelease', opacity=0.85, marginal='box',
                   labels={
                       'timeDifferenceAfterRelease': 'Time Difference After Release (years)'}
                   )
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Year Difference Between Release Date of the Movie and the Rating Date',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
num_of_rating_weekly = data.resample('m', on='timestamp').size()
fig = px.line(num_of_rating_weekly, x=num_of_rating_weekly.index, y=num_of_rating_weekly,
              labels={'y': 'Number of Ratings given',
                      'timestamp': 'Months'})
fig.update_layout(title_text='Number of Ratings given by the Users Monthly',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.update_traces(line=dict(width=3))
fig.show()

In [ ]:
unique_title = data.drop_duplicates(['title'])  # Dropping duplicate movie titles
unique_title = unique_title.movie_year.value_counts().reset_index().sort_values('index')  # Count of each years movie frequency
fig = px.line(unique_title, x='index', y='movie_year',
              labels={
                  "index": "Movie Release Year",
                  'movie_year': 'Number of Movies Released'})
fig.update_layout(title_text='Number of Movies Released each Year',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show() #Number of Movies Released each Year¶

In [ ]:
unique_title = data.drop_duplicates(['title'])  # Dropping duplicate movie titles
unique_title = unique_title.movie_year.value_counts().head(25)  # Count of each years movie frequency
fig = px.bar(unique_title, x=unique_title.index, y=unique_title, text=unique_title,
             labels={
                 "index": "Movie Release Year",
                 'y': 'Number of Movies Released'},
             color=unique_title
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 25 Years with the most Number of Movies Released',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
genre_vs_rating = data.groupby(['genres', 'rating']).size().unstack().fillna(0)
year_vs_rating = data.groupby(['movie_year', 'rating']).size().unstack().fillna(0)
movie_vs_rating = data.groupby(['title', 'rating']).size().unstack().fillna(0)

In [ ]:
# Let's calculate the Weighted Average for dataframe rows
def Weighted_Average(df):
    x = []
    for i in range(0, df.shape[0]):
        x.append((np.average(df.iloc[i].index, weights=df.iloc[i].values, axis=0)).round(2))
    return x

# Weighted Average calculation for each movie_vs_rating rows
movie_vs_rating['weightedAverage'] = Weighted_Average(movie_vs_rating)
movie_vs_rating.sort_values('weightedAverage', ascending=False).head()

# Weighted Average calculation for each year_vs_rating rows
year_vs_rating['weightedAverage'] = Weighted_Average(year_vs_rating)

# Weighted Average calculation for each genre_vs_rating rows
genre_vs_rating['weightedAverage'] = Weighted_Average(genre_vs_rating)

In [ ]:
fig = px.bar(movie_vs_rating, x=movie_vs_rating['weightedAverage'].nlargest(15).index,
             y=movie_vs_rating['weightedAverage'].nlargest(15),
             text=movie_vs_rating['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=movie_vs_rating['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Movie with the highest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
fig = px.bar(movie_vs_rating, x=movie_vs_rating['weightedAverage'].nsmallest(15).index,
             y=movie_vs_rating['weightedAverage'].nsmallest(15),
             text=movie_vs_rating['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=movie_vs_rating['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Movie with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
fig = px.bar(year_vs_rating, x=year_vs_rating['weightedAverage'].nlargest(15).index,
             y=year_vs_rating['weightedAverage'].nlargest(15),
             text=year_vs_rating['weightedAverage'].nlargest(15),
             labels={
                 "x": "Years",
                 'y': 'Weighted Rating Averages'},
             color=year_vs_rating['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Years with the highest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
fig = px.bar(year_vs_rating, x=year_vs_rating['weightedAverage'].nsmallest(15).index,
             y=year_vs_rating['weightedAverage'].nsmallest(15),
             text=year_vs_rating['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Years",
                 'y': 'Weighted Rating Averages'},
             color=year_vs_rating['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Years with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
fig = px.bar(genre_vs_rating, x=genre_vs_rating['weightedAverage'].nlargest(15).index,
             y=genre_vs_rating['weightedAverage'].nlargest(15),
             text=genre_vs_rating['weightedAverage'].nlargest(15),
             labels={
                 "x": "Genres",
                 'y': 'Weighted Rating Averages'},
             color=genre_vs_rating['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Genres with the highest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
fig = px.bar(genre_vs_rating, x=genre_vs_rating['weightedAverage'].nsmallest(15).index,
             y=genre_vs_rating['weightedAverage'].nsmallest(15),
             text=genre_vs_rating['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Genres",
                 'y': 'Weighted Rating Averages'},
             color=genre_vs_rating['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Genres with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
# Creating a new DataFrame for unique movies with their weightedAverages and Genres
_ = data.merge(movie_vs_rating.reset_index()).drop_duplicates('title')[['title', 'genres', 'weightedAverage']]
_.head()

,title,genres,weightedAverage
0,Braveheart,Action|Drama|War,4.02
212,"Basketball Diaries, The",Drama,3.37
231,"Godfather, The",Crime|Drama,4.54
423,"Godfather: Part II, The",Crime|Drama,4.34
563,Dead Poets Society,Drama,3.97


In [ ]:
drama_movies = _[_.genres.str.contains('Drama')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(drama_movies, x=drama_movies['weightedAverage'].nlargest(15).index,
             y=drama_movies['weightedAverage'].nlargest(15),
             text=drama_movies['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=drama_movies['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Drama Movies with the largest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
drama_movies = _[_.genres.str.contains('Drama')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(drama_movies, x=drama_movies['weightedAverage'].nsmallest(15).index,
             y=drama_movies['weightedAverage'].nsmallest(15),
             text=drama_movies['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=drama_movies['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Drama Movies with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
action_movies = _[_.genres.str.contains('Action')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(action_movies, x=action_movies['weightedAverage'].nlargest(15).index,
             y=action_movies['weightedAverage'].nlargest(15),
             text=action_movies['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=action_movies['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Action Movies with the largest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
sci_fi_movies = _[_.genres.str.contains('Sci-Fi')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(sci_fi_movies, x=sci_fi_movies['weightedAverage'].nlargest(15).index,
             y=sci_fi_movies['weightedAverage'].nlargest(15),
             text=sci_fi_movies['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=sci_fi_movies['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Sci-Fi Movies with the largest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
sci_fi_movies = _[_.genres.str.contains('Sci-Fi')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(sci_fi_movies, x=sci_fi_movies['weightedAverage'].nsmallest(15).index,
             y=sci_fi_movies['weightedAverage'].nsmallest(15),
             text=sci_fi_movies['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=sci_fi_movies['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Sci-Fi Movies with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
# Created a (movieId: title) dictionary for all movieId's for replacing them with their names
movieId_dict = data.drop_duplicates('title')[['movieId', 'title']].set_index('movieId').to_dict()['title']

# First 5 elements of this dictionary
list(movieId_dict.items())[:5]

[(110, 'Braveheart'),
 (147, 'Basketball Diaries, The'),
 (858, 'Godfather, The'),
 (1221, 'Godfather: Part II, The'),
 (1246, 'Dead Poets Society')]

In [ ]:
# Creating a pivot table that has indexes as user ratings, and columns as each movie title
dataRecommendation = data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Replacing dataRecommendation columns with the movie titles
dataRecommendation.columns = dataRecommendation.columns.map(movieId_dict)

dataRecommendation.head(10)

movieId,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Sudden Death,GoldenEye,"American President, The",...,Spectre,The Man from U.N.C.L.E.,The Revenant,Spotlight,Bridge of Spies,"Big Short, The",Zootopia,Arrival,Rogue One: A Star Wars Story,Logan
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def encode_units(k):
    if k <= 0:
        return 0
    if k >= 1:
        return 1


sets = dataRecommendation.applymap(encode_units)
sets.head()

movieId,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Sudden Death,GoldenEye,"American President, The",...,Spectre,The Man from U.N.C.L.E.,The Revenant,Spotlight,Bridge of Spies,"Big Short, The",Zootopia,Arrival,Rogue One: A Star Wars Story,Logan
userId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Applying Apriori algorithm to matrix that we created before (userId-movieId) and setting min support as 0.15
frequent_itemsets = apriori(sets, min_support=0.15, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.75).sort_values('lift', ascending=False)

rules = rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']]
rules.head()

,antecedents,consequents,support,lift,confidence
304,(Lord of the Rings: The Fellowship of the Ring...,"(Lord of the Rings: The Two Towers, The)",0.153937,4.365641,0.861842
305,"(Lord of the Rings: The Two Towers, The)",(Lord of the Rings: The Fellowship of the Ring...,0.153937,4.365641,0.779762
307,"(Lord of the Rings: The Return of the King, The)","(Lord of the Rings: The Two Towers, The, Lord ...",0.153937,4.304286,0.748571
302,"(Lord of the Rings: The Two Towers, The, Lord ...","(Lord of the Rings: The Return of the King, The)",0.153937,4.304286,0.885135
301,"(Lord of the Rings: The Return of the King, The)","(Matrix, The, Lord of the Rings: The Fellowshi...",0.150411,4.041855,0.731429


In [ ]:
# I will use NearestNeighbors algorithm that I learnt from the scikit-learn documentation here
knn = NearestNeighbors(n_neighbors=11, metric='cosine', algorithm='brute', n_jobs=-1)
knn.fit(dataRecommendation.values.T)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=11)

In [ ]:
# Here is our recommendations for Blade Runner, there will be 7 movie recommendations
recommendation_result = list(knn.kneighbors([dataRecommendation['Blade Runner'].values], 8))

recommendation_result  # As you can see from the results, we obtained cosine angles in the first array, and the second array gives us the dataRecommendation column order, I need to convert it to more readable form

[array([[0.        , 0.33768417, 0.36900667, 0.41210806, 0.41689512,
         0.43267537, 0.44507182, 0.45057412]]),
 array([[254, 463, 483, 352, 451, 447, 457, 430]])]

In [ ]:
# Here is our recommendations for Blade Runner, there will be 7 movie recommendations
recommendation_result = list(knn.kneighbors([dataRecommendation['Blade Runner'].values], 8))

recommendation_result  # As you can see from the results, we obtained cosine angles in the first array, and the second array gives us the dataRecommendation column order, I need to convert it to more readable form

[array([[0.        , 0.33768417, 0.36900667, 0.41210806, 0.41689512,
         0.43267537, 0.44507182, 0.45057412]]),
 array([[254, 463, 483, 352, 451, 447, 457, 430]])]

Some Movie Recommendations

In [ ]:
# Movie Recommendation as Function

def movie_recommendation(movie_name, num_of_recommendations):
    a = dataRecommendation.columns.to_frame().reset_index(drop=True).to_dict()['movieId']
    recommendation_result = list(knn.kneighbors([dataRecommendation[movie_name].values], num_of_recommendations + 1))
    recommendation_result = pd.DataFrame(np.vstack((recommendation_result[1], recommendation_result[0])),
                                         index=['movieId', 'Cosine_Similarity (degree)']).T
    recommendation_result = recommendation_result.drop([0]).reset_index(drop=True)
    recommendation_result.movieId = recommendation_result.movieId.map(a)
    return recommendation_result

In [ ]:
movie_recommendation('Amazing Spider-Man, The', 7)

,movieId,Cosine_Similarity (degree)
0,Monsters University,0.408173
1,Man of Steel,0.408345
2,"Hobbit: An Unexpected Journey, The",0.434187
3,Divergent,0.454666
4,"Avengers, The",0.466670
5,Despicable Me 2,0.472122
6,Thor,0.474719


In [ ]:
movie_recommendation('Mulan', 7)

,movieId,Cosine_Similarity (degree)
0,Tarzan,0.517641
1,Hercules,0.524953
2,"Little Mermaid, The",0.533420
3,"Emperor's New Groove, The",0.572753
4,Lilo & Stitch,0.579825
5,Cinderella,0.582846
6,Antz,0.591699


In [ ]:
movie_recommendation('Final Destination', 7)

,movieId,Cosine_Similarity (degree)
0,Final Destination 2,0.495401
1,How the Grinch Stole Christmas,0.622796
2,"Mummy Returns, The",0.665880
3,"Patriot, The",0.668160
4,Bridesmaids,0.668623
5,"Cell, The",0.681534
6,Stir of Echoes,0.681964


In [ ]:
movie_recommendation('Shining, The', 7)

,movieId,Cosine_Similarity (degree)
0,Alien,0.443175
1,"Exorcist, The",0.451497
2,Reservoir Dogs,0.453357
3,Blade Runner,0.474606
4,Aliens,0.482768
5,Full Metal Jacket,0.482997
6,Seven,0.486232


In [ ]:
movie_recommendation('Pulp Fiction', 7)

,movieId,Cosine_Similarity (degree)
0,"Silence of the Lambs, The",0.309315
1,Seven,0.323338
2,"Usual Suspects, The",0.361061
3,"Shawshank Redemption, The",0.365156
4,Forrest Gump,0.396412
5,Reservoir Dogs,0.399704
6,Terminator 2: Judgment Day,0.403482


In [ ]:
movie_recommendation("Reservoir Dogs", 7)

,movieId,Cosine_Similarity (degree)
0,Full Metal Jacket,0.385580
1,Pulp Fiction,0.399704
2,Goodfellas,0.401120
3,Seven,0.408734
4,"Clockwork Orange, A",0.440372
5,Fargo,0.440470
6,Memento,0.443695


In [ ]:
movie_recommendation('Batman Returns', 7)

,movieId,Cosine_Similarity (degree)
0,RoboCop,0.484975
1,Total Recall,0.548788
2,Superman,0.554005
3,Barbarella,0.567526
4,Crocodile Dundee,0.568240
5,Legend,0.570406
6,Highlander,0.575476


In [ ]:
movie_recommendation('Logan', 7)

,movieId,Cosine_Similarity (degree)
0,Arrival,0.396308
1,Rogue One: A Star Wars Story,0.540300
2,Doctor Strange,0.554179
3,Nightcrawler,0.571643
4,Zootopia,0.581081
5,About Time,0.605077
6,Predestination,0.609939


In [ ]:
movie_recommendation('Matrix, The', 7)

,movieId,Cosine_Similarity (degree)
0,"Sixth Sense, The",0.350435
1,"Lord of the Rings: The Fellowship of the Ring,...",0.358814
2,Fight Club,0.362804
3,"Lord of the Rings: The Return of the King, The",0.363450
4,Star Wars: Episode IV - A New Hope,0.367426
5,Star Wars: Episode V - The Empire Strikes Back,0.378461
6,"Fifth Element, The",0.385570


In [ ]:
movie_recommendation('Casablanca', 7)

,movieId,Cosine_Similarity (degree)
0,Rear Window,0.498622
1,North by Northwest,0.504909
2,Citizen Kane,0.507358
3,"Godfather, The",0.519606
4,Annie Hall,0.523457
5,"Graduate, The",0.541232
6,"Godfather: Part II, The",0.550557


In [ ]:
movie_recommendation("Harry Potter and the Sorcerer's Stone", 7)

,movieId,Cosine_Similarity (degree)
0,Harry Potter and the Chamber of Secrets,0.241394
1,Harry Potter and the Prisoner of Azkaban,0.303359
2,Harry Potter and the Goblet of Fire,0.330163
3,Pirates of the Caribbean: The Curse of the Bla...,0.416384
4,Spider-Man,0.421671
5,Shrek,0.424650
6,Harry Potter and the Order of the Phoenix,0.442152


In [ ]:
movie_recommendation('Rain Man', 7)

,movieId,Cosine_Similarity (degree)
0,Good Will Hunting,0.514210
1,Ghostbusters,0.514829
2,"Truman Show, The",0.515690
3,E.T. the Extra-Terrestrial,0.515861
4,Die Hard,0.523006
5,Groundhog Day,0.525702
6,American Beauty,0.532534
